### Efficient phosphate remediation using existing wastewater treatment plant technology

Demian Riccardi, Cameron Van Cleave, Andrew Hood, Corinne C. Deibel, Mike Deibel

    Department of Chemistry
    Earlham College
    Richmond, IN
    
#### Acknowledgements:

    Jamey Pavey and the Center for Integrated Learning
    Pat Smoker and Tim Orrill and the Richmond Sanitary District

In [2]:
use Modern::Perl;
use List::Util qw (sum);
use PDL::Lite;
use PDL::Stats;
my @data = qw(0.50824
0.788
0.14144
0.832
1.11824
1.2956
0.71264
0.56384
1.67784
0.92624 );
my $pdl = pdl(@data);
say $pdl->stdv;
my $avg_tp = sum (@data)/ scalar(@data);

0.410489162166311


0.856408


### Two functions for interconverting TP/flow-rate and kilograms per day/flow-rate 
- kilgrams per day (KgD)
- total phosphate (TP)
- TP concentrations are in ppm and the flow-rates are in MGD.

In [3]:
sub tp_kgd{
    # calculate KgD from MGD and TP concentration
    my $tp        = shift; # in ppm
    my $flow_rate = shift; # in MGD

  # 1 ppm corresponds to 1 mg / 1 L using the density of water 1 g / 1 ml
    my $lpd = $flow_rate * 3.78541E6; 
    my $kgd = $tp*$lpd*1E-6;
    return ($kgd);
}

sub kgd_tp{
    # calculate MGD from KgD and  from MGD and TP concentration
    my $kgd = shift; # 
    my $flow_rate = shift; # in MGD


  # 1 ppm corresponds to 1 mg / 1 L using the density of water 1 g / 1 ml
    my $lpd = $flow_rate * 3.78541E6; 
    my $tp  = $kgd = $kgd/$lpd*1E6;
    return ($kgd);
}

sub mix_water {
    my ($tp_a,$flow_a,$tp_b,$flow_b) = @_;
    my $flow_mix = $flow_a + $flow_b;
    my $tp_mix   = ($tp_a*$flow_a + $tp_b*$flow_b)/$flow_mix;
    return ($tp_mix, $flow_mix);
}

## N treatment plants working in parallel
Here is a subroutine that calculates the new TP and flow-rate of the river as a result of N treatment plants working in parallel. Depending on the parameters of the treatment plant, it may treat municipal waste and add to the flow of the river or it may only pull in from the river and not affect the flow.

In [4]:
sub river_Nwwtp {
  # the flow and TP just up and just down river from WWTP
  # allow parameter N to scale-up treatment in parallel
  # returns a hash with river info in it
  
  my ($river,$wwtp,$N) = (shift,shift,shift);
  $N = 1 unless defined($N);
  #flow-rate upstream and flow-rate downstream of treatment plant
  
  my $river_flow = $river->{flow_rate} - $N*$wwtp->{pull} ;
  my $river_tp   = $river->{tp};

  my $wwtp_flow = $N*($wwtp->{push}+$wwtp->{pull});
  my $wwtp_tp   = $wwtp->{tp};
  
  my ($new_tp, $new_flow) = mix_water($river_tp, $river_flow, $wwtp_tp, $wwtp_flow); 

  return({flow_rate => $new_flow, tp => $new_tp});
  
}



#### Let's define a couple of rivers and a couple of WWTPs to make some comparisons

In [5]:
use Data::Dumper;

my $river_high = {
    flow_rate => 93,      # MGD
    tp        => $avg_tp, # ppm
    #tp        => 1.0,
};

my $river_low = {
    flow_rate => 20,      # MGD
    tp        => $avg_tp, # ppm
};

my $WWTP_def = {
       pull => 0,
       push => 5,
       tp   => 0.1,
};

my $WWTP_pull = {
       pull => 7.5,
       push => 5,
       tp   => 0.1,
};

my $WWTP_epa = {
       pull => 0,
       push => 5,
       tp   => 0.01,
};


my $dh = river_Nwwtp($river_high,$WWTP_def);
my $hkgd = tp_kgd($river_high->{tp},$river_high->{flow_rate});
printf ("high_flow Kg per day: %10.2f\n", $hkgd);

my $dl = river_Nwwtp($river_low,$WWTP_def);
my $lkgd = tp_kgd($river_low->{tp},$river_low->{flow_rate});
printf ("low_flow  Kg per day: %10.2f\n", $lkgd);

my $def_wwtp_kgd = tp_kgd($WWTP_def->{tp},$WWTP_def->{push});
printf ("Def WWTP  Kg per day: %10.2f\n", $def_wwtp_kgd);

my $epa_wwtp_kgd = tp_kgd($WWTP_epa->{tp},$WWTP_epa->{push});
printf ("EPA WWTP  Kg per day: %10.2f\n", $epa_wwtp_kgd);
 
print Dumper $dh, $dl;

foreach ($river_high,$river_low){

    my $def  = river_Nwwtp($_,$WWTP_def);

    my $pull = river_Nwwtp($_,$WWTP_pull);
    my $hperc_red = 100*($def->{tp}-$pull->{tp})/$def->{tp};
    printf ("Relative Reduction Pull: %5.2f\n", $hperc_red ); 
    
    my $epa = river_Nwwtp($_, $WWTP_epa);
    my $lperc_red = 100*($def->{tp}-$epa->{tp})/$def->{tp} ;
    printf ("Relative Reduction EPA:  %5.2f\n", $lperc_red );
   # print Dumper $epa
}

high_flow Kg per day:     301.49
low_flow  Kg per day:      64.84
Def WWTP  Kg per day:       1.89
EPA WWTP  Kg per day:       0.19
$VAR1 = {
          'tp' => '0.817815755102041',
          'flow_rate' => 98
        };
$VAR2 = {
          'tp' => '0.7051264',
          'flow_rate' => 25
        };
Relative Reduction Pull:  7.08
Relative Reduction EPA:   0.56
Relative Reduction Pull: 32.18
Relative Reduction EPA:   2.55


### Let's do some analysis to compare the EPA to the Pulling strategy

In [6]:
my $df = 0.1;
my (@rppm, @epa_red, @pull_red);
foreach my $ippm (1 .. 20){
    my $rppm = $ippm * $df;
    my $river = {
        flow_rate => 50,      # MGD
        tp        => $rppm,  # ppm
    };
    
    my $def  = river_Nwwtp($river,$WWTP_def);
    my $pull = river_Nwwtp($river,$WWTP_pull);
    my $pull_red = 100*($def->{tp}-$pull->{tp})/$def->{tp};    
    my $epa = river_Nwwtp($river, $WWTP_epa);
    my $epa_red = 100*($def->{tp}-$epa->{tp})/$def->{tp} ;
    printf ("%10.2f %10.2f %10.2f \n", $rppm, $epa_red, $pull_red);
    push @rppm, $rppm;
    push @epa_red, $epa_red;
    push @pull_red, $pull_red;
    
}

      0.10       8.18       0.00 
      0.20       4.29       7.14 
      0.30       2.90       9.68 
      0.40       2.20      10.98 
      0.50       1.76      11.76 
      0.60       1.48      12.30 
      0.70       1.27      12.68 
      0.80       1.11      12.96 
      0.90       0.99      13.19 
      1.00       0.89      13.37 
      1.10       0.81      13.51 
      1.20       0.74      13.64 
      1.30       0.69      13.74 
      1.40       0.64      13.83 
      1.50       0.60      13.91 
      1.60       0.56      13.98 
      1.70       0.53      14.04 
      1.80       0.50      14.09 
      1.90       0.47      14.14 
      2.00       0.45      14.18 


In [7]:
IPerl->load_plugin( "ChartClicker" );
my $cc = Chart::Clicker->new(width => 523, height => 384);

my $epa = Chart::Clicker::Data::Series->new(
  keys    => [@rppm],
  values  => [@epa_red],
);

my $pull = Chart::Clicker::Data::Series->new(
  keys    => [@rppm],
  values  => [@pull_red],
);

# build the dataset
my $dataset = Chart::Clicker::Data::DataSet->new(
    series  =>      [ $epa, $pull ],
);

$cc->add_to_datasets($dataset);

$cc;

The percent reduction in TP is shown on the y-axis.  The concentration of TP (ppm) of the river is shown on the x-axis. 

* Series 0: Treating 5 MGD from the city and outputting 5 MGD at 0.035 ppm to the river.
* Series 1: Pulling 7.5 MGD from river and mixing with 5 MGD from the city.  Outputting 12.5 MGD into river at 0.1 ppm.

In [8]:
print Dumper $river;

$VAR1 = {
          'tp' => '0.1',
          'flow_rate' => 50
        };


1


In [9]:
print Dumper $WWTP_def;

$VAR1 = {
          'pull' => 0,
          'push' => 5,
          'tp' => '0.1'
        };


1


In [10]:
print Dumper $WWTP_epa;

$VAR1 = {
          'tp' => '0.01',
          'pull' => 0,
          'push' => 5
        };


1


In [11]:
print Dumper $WWTP_pull;

$VAR1 = {
          'tp' => '0.1',
          'pull' => '7.5',
          'push' => 5
        };


1
